# Домашнее задание 1. Сравнение стилей

## Работу выполнили Ковалев Евгений и Сухарев Иван, 4 группа

В данном домашнем задании мы проверяли гипотезу о различии характеров распределений частот частей речи в текстах разных стилей. Для 
этого мы составили коллекции текстов официально-делового, публицистического, разговорного и художественного стилей. Далее с помощью 
морфологического процессора из библиотеки pymorphy2 мы определили части речи, к которым относятся слова из каждой коллекции, и 
составили для них частотные словари. Затем для каждой пары наборов частот был посчитан коэффициент корреляции Спирмена. Исходя из результатов, можно сделать вывод, что гипотеза не подтверждается, потому что корреляция между частотными словарями для разных стилей текстов достаточно высокая. Следует заметить, что полученные значения коэффициентов корреляции отличаются друг от друга, и, например, официально-деловой стиль больше похож на публицистический, чем на художественный.

## Создание коллекций

*[2 балла] Составьте самостоятельно как минимум две коллекции
текстов разных стилей (например, коллекция текстов в публицистическом
стиле и коллекция текстов в научном стиле). Коллекции текстов
должны быть достаточно большие (порядка 5000 токенов). Посчитайте
количество токенов и типов в каждой коллекции.*

In [1]:
import re
import glob
import nltk
from pymorphy2 import MorphAnalyzer
import pandas as pd
from nltk.tokenize import WhitespaceTokenizer
from string import punctuation

Зададим сортировку.

In [2]:
numbers = re.compile(r'(\d+)')
def numericalSort(value):
    parts = numbers.split(value)
    parts[1::2] = map(int, parts[1::2])
    return parts

Токенизация.

In [3]:
exclude = set(punctuation + '0123456789'+u'–—'+u'«»')
def tokenization(collection):
    tokens = WhitespaceTokenizer().tokenize(collection.lower())
    types = nltk.FreqDist(tokens)
    return tokens, types

Создаём коллекции из текстов.

In [1]:
def create_collection_with_range(style, ran):
    texts = []
    for file in sorted(glob.glob(style + '\\*.txt'), key=numericalSort):
        texts.append(file)
    collection = []
    for i in range(ran):
        lines = open(texts[i]).readlines()
        collection.append(' '.join(''.join(lines).replace('\n', ' ').split()))
    collection = ' '.join(collection)
    collection = ''.join(ch for ch in collection if ch not in exclude)
    return collection

In [5]:
colloquial = create_collection_with_range('colloquial', 15)
publicistic = create_collection_with_range('publicistic', 15)
art = create_collection_with_range('art', 5)
official = create_collection_with_range('official', 15)

Выделяем токены и типы, считаем их количество.

In [6]:
colloquial_tokens, colloquial_types = tokenization(colloquial)
publicistic_tokens, publicistic_types = tokenization(publicistic)
art_tokens, art_types = tokenization(art)
official_tokens, official_types = tokenization(official)

In [7]:
print('number of tokens (colloquial): ', len(colloquial_tokens))
print('number of types (colloquial):', len(colloquial_types))
print('number of tokens (publicistic): ', len(publicistic_tokens))
print('number of types (publicistic):', len(publicistic_types))
print('number of tokens (art): ', len(art_tokens))
print('number of types (art):', len(art_types))
print('number of tokens (official): ', len(official_tokens))
print('number of types (official):', len(official_types))

number of tokens (colloquial):  16756
number of types (colloquial): 6039
number of tokens (publicistic):  6753
number of types (publicistic): 3048
number of tokens (art):  16953
number of types (art): 5694
number of tokens (official):  15363
number of types (official): 4712


## Определение частей речи и составление частотных словарей

*[5 баллов] Используя любой морфологический процессор, который
вам нравится (pymorphy2, mystem), определите к какой части речи
относятся слова из каждой коллекции текстов. При помощи nltk.FreqDist()
составьте частотные словари: часть речи – количество слов, к ней
относящихся.*


Составляем частотные словари.

In [8]:
morph = MorphAnalyzer()
def morphanalysis(collection):
    tags = []
    for word in collection.split():
        tags.append(morph.parse(word)[0].tag.POS)
    tags_dict = nltk.FreqDist(tags)
    return tags_dict

In [9]:
colloquial_tags = morphanalysis(colloquial)
publicistic_tags = morphanalysis(publicistic)
art_tags = morphanalysis(art)
official_tags = morphanalysis(official)

Выведем частоты частей речи в порядке убывания.

In [10]:
print('Colloquial:')
for i in sorted(colloquial_tags, key=colloquial_tags.get, reverse=True):
    print(i, colloquial_tags[i])

Colloquial:
NOUN 4178
ADJF 2077
CONJ 1915
VERB 1854
PREP 1821
NPRO 1252
ADVB 1099
PRCL 1030
INFN 660
ADJS 185
None 124
PRTS 112
PRTF 110
PRED 106
COMP 98
GRND 58
INTJ 49
NUMR 28


In [11]:
print('Publicistic:')
for i in sorted(publicistic_tags, key=publicistic_tags.get, reverse=True):
    print(i, publicistic_tags[i])

Publicistic:
NOUN 2960
PREP 1022
ADJF 724
VERB 661
CONJ 339
ADVB 209
PRCL 162
INFN 136
NPRO 133
None 114
PRTF 80
NUMR 56
ADJS 51
PRTS 50
GRND 19
COMP 14
PRED 13
INTJ 10


In [12]:
print('Art:')
for i in sorted(art_tags, key=art_tags.get, reverse=True):
    print(i, art_tags[i])

Art:
NOUN 4243
VERB 2305
CONJ 1942
PREP 1608
ADJF 1587
NPRO 1433
ADVB 1234
PRCL 977
INFN 417
GRND 261
None 211
PRTF 173
ADJS 167
PRED 102
NUMR 86
INTJ 71
COMP 69
PRTS 67


In [13]:
print('Official:')
for i in sorted(official_tags, key=official_tags.get, reverse=True):
    print(i, official_tags[i])

Official:
NOUN 6771
ADJF 2752
PREP 2017
CONJ 1201
VERB 697
PRTF 423
ADVB 300
INFN 279
PRCL 241
NPRO 212
None 184
PRTS 103
ADJS 59
NUMR 53
PRED 33
INTJ 14
COMP 12
GRND 12


## Корреляции Спирмена

*[3 балла] Посчитайте коэффициент корреляции Спирмена для
полученных на предыдущем шаге частот частей речи. На основании
полученного значения, сделайте вывод: подтверждается ли гипотеза,
сформулированная в задании? Если вы рассматривали больше
двух стилей, можно ли утверждать, что один стиль больше похож
на второй, чем на третий?*

Для удобства вывода загрузим данные в pandas и воспользуемся встроенной функцией corr().

In [16]:
listOfTags = list(set(colloquial_tags))
tags = [art_tags, official_tags, colloquial_tags, publicistic_tags]
vals = []
for i in range(4):
    newList = []
    for name in listOfTags:
        newList.append(tags[i][name])
    vals.append(newList)
    
df = pd.DataFrame(data=vals).T
df.columns = ['Art', 'Offic', 'Colloq', 'Public']
df.corr('spearman')

,Art,Offic,Colloq,Public
Art,1.000000,0.826020,0.886481,0.902993
Offic,0.826020,1.000000,0.887971,0.937532
Colloq,0.886481,0.887971,1.000000,0.905057
Public,0.902993,0.937532,0.905057,1.000000
